In [1]:
# akses ke google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#ekstrak dataset
import zipfile
dataset_path='/content/drive/MyDrive/dataset/wajah.zip'
ektraksi='/content/dataset'

with zipfile.ZipFile(dataset_path,'r') as zip_ref:
    zip_ref.extractall(ektraksi)

In [11]:
# %%
# Install necessary libraries
!pip install opencv-python matplotlib numpy

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import files # Import files module for file upload

# Definisikan path ke folder dataset
dataset_folder = '/content/dataset/Celebrity Faces Dataset'

# diibuat mencari dan menampilkan gambar serupa
def find_and_display_similar(query_img, dataset_folder, num_to_display):
    print("\nMencari gambar yang mirip dengan gambar query yang diunggah.")

    # Hitung histogram gambar query (menggunakan 3 channel BGR)
    # Ukuran bin [8, 8, 8] untuk B, G, R
    # Range [0, 256, 0, 256, 0, 256] untuk B, G, R
    query_hist = cv2.calcHist([query_img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    # Normalisasi histogram
    cv2.normalize(query_hist, query_hist, 0, 1.0, cv2.NORM_MINMAX)

    # Dictionary untuk menyimpan hasil perbandingan: {image_path: similarity_score}
    similarity_scores = {}

    # metode perbandingan histogram (contoh: CV_COMP_CORREL)(ngetong korelasi)
    comparison_method = cv2.HISTCMP_CORREL

    # Iterasi melalui semua file gambar di folder dataset
    for root, _, files_list in os.walk(dataset_folder):
        for file in files_list:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                img_path = os.path.join(root, file)

                # baca gambar saat ini
                current_img = cv2.imread(img_path)
                if current_img is None:
                    # print(f"Warning: Gagal memuat gambar dari {img_path}")
                    continue

                # Hitung histogram gambar saat ini
                current_hist = cv2.calcHist([current_img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
                # Normalisasi histogram
                cv2.normalize(current_hist, current_hist, 0, 1.0, cv2.NORM_MINMAX)

                # Bandingkan histogram query dengan data base
                similarity = cv2.compareHist(query_hist, current_hist, comparison_method)
                similarity_scores[img_path] = similarity

    # Urutkan skor kemiripan berdasarkan metode perbandingan
    if comparison_method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_INTERSECT]:
        sorted_similar_images = sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True)
    else:
        sorted_similar_images = sorted(similarity_scores.items(), key=lambda item: item[1], reverse=False)

    # Tampilkan gambar query
    print("\nGambar Query yang Diunggah:")
    cv2_imshow(query_img)

    # Tampilkan N gambar paling mirip
    print(f"\n{num_to_display} Gambar Paling Mirip Ditemukan:")

    images_to_display = sorted_similar_images[:num_to_display]

    if not images_to_display:
         print("Tidak ada gambar serupa ditemukan di dataset.")
         return

    for i, (img_path, score) in enumerate(images_to_display):
        similar_img = cv2.imread(img_path)
        if similar_img is not None:
            print(f"[{i+1}] Gambar: {img_path} (Skor Mirip: {score:.4f})")
            cv2_imshow(similar_img)
        else:
            print(f"Warning: Gagal memuat gambar paling mirip dari {img_path}")

# --- Alur Utama: Unggah Gambar dan Cari Serupa ---
uploaded = files.upload() # Meminta pengguna untuk mengunggah file

if uploaded:
    # Ambil nama file yang diunggah (kita asumsikan hanya satu file)
    uploaded_filename = list(uploaded.keys())[0]
    print(f"\nFile '{uploaded_filename}' berhasil diunggah.")

    # Baca gambar yang diunggah menggunakan cv2
    # cv2.imread membutuhkan path file, jadi kita simpan dulu ke disk
    # Meskipun bisa langsung dari memori dengan numpy dan cv2.imdecode, menyimpan ke disk lebih sederhana untuk Colab
    with open(uploaded_filename, 'wb') as f:
        f.write(uploaded[uploaded_filename])

    query_image_path_uploaded = f'/content/{uploaded_filename}'
    query_img_uploaded = cv2.imread(query_image_path_uploaded)

    if query_img_uploaded is not None:
        # Jalankan fungsi pencarian dan tampilan dengan gambar yang diunggah
        find_and_display_similar(query_img_uploaded, dataset_folder, num_to_display=2)
    else:
        print(f"Error: Gagal memuat gambar yang diunggah dari {query_image_path_uploaded}. Pastikan file yang diunggah adalah gambar yang valid.")

    # Opsional: Hapus file yang diunggah setelah selesai
    # os.remove(query_image_path_uploaded)
    # print(f"File '{uploaded_filename}' dihapus.")
else:
    print("\nTidak ada file yang diunggah. Program selesai.")

print("\nProgram selesai.")

MessageError: RangeError: Maximum call stack size exceeded.